<a href="https://colab.research.google.com/github/CharlotteY2003/Web-Scraper/blob/main/First_Web_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from time import sleep
from random import randint

In [4]:
titles = []
prep = []
cats = []
servings = []
ingreds = []

In [5]:
urls = [
        "https://www.simplyrecipes.com/quick-vegetarian-recipes-5091240",
        "https://www.simplyrecipes.com/comfort-food-recipes-5091432",
        'https://www.simplyrecipes.com/healthy-breakfast-recipes-5091517',
        'https://www.simplyrecipes.com/vegetarian-dinner-recipes-5091427',
        'https://www.simplyrecipes.com/vegetarian-breakfast-recipes-5091239'
]

# Retrieve Information from Recipe Site

In [7]:
def getRecipe(item):

  #get recipe name
  name = item.contents[1].contents[3].contents[1].text
  name = name.strip()
  titles.append(name)

  #vegetarian or not
  text = str(item.div.contents[1])
  tag = BeautifulSoup(text, 'html.parser').div
  cat = tag['data-tax-tag']
  cats.append(cat)
  
  #get prep time
  time = item.contents[3].contents[1].span.text
  time = time.strip()
  prep.append(time)

  #switch to access full recipe page
  full_r = item.parent['href']
  url = full_r
  results = requests.get(url)
  soup = BeautifulSoup(results.text, "html.parser")
  recipe_div = soup.find_all('div', class_="comp article__content structured-content structured-project-content mntl-block")
  new_item = recipe_div[0]
  
  #get recipe's serving number
  serving = getServingNum(url)
  servings.append(serving)

  #get ingredients
  ingreds.append(getFood(url))

In [8]:
def getServingNum(link):
  results = requests.get(link)
  soup = BeautifulSoup(results.text, "html.parser")
  recipe_div = soup.find_all('div', class_="loc recipe-serving project-meta__recipe-serving")
  try:
    new_item = recipe_div[0]
  except:
    recipe_div = soup.find_all('div', class_ = 'loc recipe-yield project-meta__recipe-yield')
  #print(recipe_div)
  new_item = recipe_div[0]
  #get recipe's serving number
  serving = new_item.contents[1].contents[3].text#[5]#.div.contents[3].div.contents[1].contents[3].text 
  serving = serving.replace('\n', '')
  #print(serving)
  return serving

In [9]:
#gets each ingredient from individual recipe
def getFood(link):
  temp_arr = []
  results = requests.get(link)
  soup = BeautifulSoup(results.text, "html.parser")
  recipe_div = soup.find('ul', class_='comp ingredient-list simple-list simple-list--circle ')
  try:
    food = recipe_div.contents[1]
  except:
    recipe_div = soup.find('ul', class_='structured-ingredients__list text-passage')
  
  food = recipe_div.contents[1]
  temp_arr.append(food.text.strip())

  score = 0
  for sibling in food.next_siblings:
    if (not score%2 == 0):
      #print(sibling.text)
      food = sibling.text.strip()
      temp_arr.append(food)
    score = score +1

  return temp_arr

In [10]:
#loops through all recipes on that page, extracting recipe name, vegatarian or not, prep time, 
#number of servings, and ingredients
def getAllRecipes(recipe_div):
  for container in recipe_div:
    getRecipe(container) 
    sleep(randint(2,10))

In [11]:
#creates table with given categories
def createTable():
  recipes = pd.DataFrame({
    'titles': titles,
    'category': cats,
    'Prep time': prep,
    'servings': servings,
    'Ingreds': ingreds  
}) 
  recipes.to_csv('recipes.csv')
  return recipes

In [12]:
# Gets all specified information from multiple pages (urls) of website and creates a table out of them
def scrapeRecipes(urls):
  for i in range(len(urls)):
    results = requests.get(urls[i])
    soup = BeautifulSoup(results.text, "html.parser")
    recipe_div = soup.find_all('div', class_="card__wrapper")
    getAllRecipes(recipe_div)

  recipe = createTable()
  return recipe

# Function being Called

In [13]:
scrapeRecipes(urls)

,titles,category,Prep time,servings,Ingreds
0,Socca,Quick Vegetarian,13 mins,4 servings,"[1 1/2 cups chickpea flour, 3 tablespoons oliv..."
1,Gochujang Green Beans,Quick Vegetarian,14 mins,4 servings,"[2 tablespoons gochujang, 1 tablespoon rice vi..."
2,Oat and Tahini Breakfast Cookies,Quick Vegetarian,27 mins,12 servings,"[1/4 cup pepitas, 1/4 cup walnuts, coarsely ch..."
3,Easy Avocado and Black Bean Quesadillas,Quick Vegetarian,25 mins,4 servings,"[For the avocado mash:, 2 large or 3 small avo..."
4,Chickpea Salad Sandwich,Quick Vegetarian,15 mins,2 sandwiches,"[1 can (15 ounces) chickpeas, rinsed and drain..."
...,...,...,...,...,...
153,Oat and Tahini Breakfast Cookies,Vegetarian Breakfast,27 mins,12 servings,"[1/4 cup pepitas, 1/4 cup walnuts, coarsely ch..."
154,Vegetarian Eggs Benedict with Spinach and Avocado,Vegetarian Breakfast,25 mins,4 servings,"[For the Quick Blender Hollandaise Sauce:, 2 e..."
155,Zucchini-Feta Frittata,Vegetarian Breakfast,60 mins,4to 6 servings,"[3 medium zucchini (1 1/4 pounds), 1/4 teaspoo..."
156,Scalloped Turnips,Vegetarian Breakfast,95 mins,6 servings,"[4 Tbsp butter, 1/2 cup thinly sliced onions, ..."
